In [1]:
import pandas as pd

In [2]:
#Import df
data=pd.read_csv("train.csv")
#Rename first column as "ID"
data.rename(columns={'Unnamed: 0': 'ID'}, inplace=True)

data.head(5)

,ID,date,org,tld,ccs,bcced,mail_type,images,urls,salutations,designation,chars_in_subject,chars_in_body,label
0,0,"Mon, 6 Nov 2017 11:13:45 +0100",reply,ebay.in,0,0,multipart/alternative,35,120,0,0,49.0,80027,2
1,1,"Wed, 14 Feb 2018 11:00:16 -0000",edm,efinmail.com,0,0,multipart/alternative,1,7,0,0,107.0,2961,1
2,2,"Wed, 6 Jul 2016 19:53:37 +0000",usebackpack,com,0,0,text/html,4,17,0,0,35.0,25149,1
3,3,"Fri, 11 Oct 2019 11:25:40 +0200",granular,ai,0,0,multipart/mixed,0,0,0,0,15.0,635296,1
4,4,"Tue, 07 Nov 2017 11:07:18 +0000 (UTC)",github,com,1,0,multipart/alternative,2,11,0,0,49.0,2355,1


In [3]:
#Store all the mail types and merge duplicates in dummy variables
all_type=pd.get_dummies(data["mail_type"])

all_type["text/html"] = all_type["text/html"]+all_type["text/html "]+all_type["Text/Html"]
all_type=all_type.drop(["text/html ","Text/Html"], axis=1)

all_type["multipart/mixed"] = all_type["multipart/mixed"]+all_type["Multipart/Mixed"]
all_type=all_type.drop("Multipart/Mixed", axis=1)

all_type.shape


In [4]:
#Store org/tld in dummy variables
all_org=pd.get_dummies(data["org"])
all_tld=pd.get_dummies(data["tld"])

In [5]:
#Merge data

input=data.drop(['ID','date','org','mail_type','tld','label'], axis=1)

input=pd.concat([input,all_type,all_org,all_tld],axis=1)
target=pd.get_dummies(data["label"])

In [ ]:
#Going crazy with DL

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import KFold
import numpy as np



# Cross-Val fitting
fold_no = 1
acc_per_fold = []
loss_per_fold = []

inputs = input.to_numpy()
targets = target.to_numpy()

for test, train in KFold(n_splits=3, shuffle=True).split(inputs, targets):
    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')


    ## Model building

    model = keras.Sequential()
    model.add(keras.Input(shape=(len(inputs[0]),)))
    model.add(layers.Dense(100, activation="sigmoid"))
    model.add(layers.Dense(100, activation="sigmoid"))
    # model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(8, activation="softmax"))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


    history = model.fit(inputs[train], targets[train],
                  batch_size=5,
                  epochs=5,
                  verbose=1)
    
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    fold_no += 1


# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

"""
score = model.evaluate(input_test, target_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')
"""